<a href="https://colab.research.google.com/github/ANILESLAVATH12/customer-churn/blob/main/Predict_Customer_Churn_Case_Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Case Study: Predict Customer Churn Using Machine Learning
Prepared by Michelle Bonat

## 1: Frame the Problem

## **Case Study Overview**

Many companies would benefit immensely by investigating their customer churn and taking steps to improve it. Why? Simply stated, it usually takes much less resources to keep a customer than to get a new one. So if you have the information you need on why customers are leaving (churning) you can use this proactively to reduce your churn. Let's look at how we can develop this intelligence using machine learning.

----
##About The Data and The Approach

We're told by our colleagues at the hypothetical company that customer churn is at 50% within 3 months. That means that within 3 months of a set of customers that sign up for the paid product, by the end of 3 months half of them will have cancelled. This is an urgent problem we need to help fix with machine learning!

## - Dataset:
For this dataset I used an IBM Sample Dataset for customer churn from the TelCo industry which can be found [here](https://community.watsonanalytics.com/wp-content/uploads/2015/03/WA_Fn-UseC_-Telco-Customer-Churn.csv)


## - Project Steps:

1. **Talk with stakeholders (S/H) to: **
  * **Discuss/determine goals and metrics for the project.**  Is reducing churn the right problem to solve? How specifically shall we measure performance improvement? What is our success goal? (Assume here: it is the right problem, we measure performance overall by reducing customer churn, success is reducing customer churn by 10% in next 6 months).
  * **Understand what deliverables are useful for internal stakeholders** (Assume it is churn prediction factors, later a spreadsheet of customer churn predictions, production pipeline and perhaps an internal dashboard).
  * **Discuss an iterative approach with timing and deliverables**, at a high level. First see if the data is predictive. Set a goal for what predictive means (Assume here: 80% +). Come back with findings in 2 weeks.
2. **Work with SME's (Subject Matter Experts) to come up with a hypothesis.** Here we'll assume that tenure, addt'l products/features, tech support interactions are key; also possibly how they pay.
3. **Get the data.** Work with data engineering to get the data, if needed.
3. **Talk to my team and resource the project.** Include team personal skills development where possible. In this case for example purposes, assume all the team is booked and I'll do the initial analysis which appears here.
3. **Prototype some approaches.** Is the data usable? Can churn be reliably predicted? Do we need other data? Can it eventually be done at scale?
4. **When a valid prototype is achieved, begin A/B testing.** If the prototype indicates a workable approach is viable, coordinate with S/H and team on next steps. Put A/B testing in place.
5. **Iterate data and models learned from A/B testing and internal model testing.** Get to target success metrics on prediction rate and any other target metrics in order to move forward.
6. **Create the production pipeline. **Once an initial model prototype has been established that surpasses the desired performance thresholds, it is ready for the production line.
  * Start simple with incremental wins. Expland as it makes sense with priorities and resources.
    * Load the flagged users (potential churners) back to the database as a first step
    * Eventually, move to automated data injection from a data warehouse
    * Refine the model and the data
    * Report performance to stakeholders and team on an ongiong basis. Get feedback and discuss next steps. Implement, learn, improve, repeat.
7. **Suggest some ways to improve churn in products and processes based on learnings from the activities listed above.**
    * These notes are listed at the bottom of this notebook.






## 2: Data we would we like to have:

Generally speaking we would like to have as much data as possible with a long history over as many categories as possible.

Here are some types of data that are useful in customer churn analysis:
* Customer ID or other identification information
* Date the customer was acquired
* How the customer was acquired (source of sale i.e. referral, web signup, etc.)
* Plan type (what subscription they are on)
* Cohort analysis by user type (seasonal onboards by marketing campaign or time of year, etc.)
* If they use add ons (sush as Online Security or Device Protection)?
* Have they set up to pay for their subscription online?
* Customer size
* Customer segment type (i.e. company user, accountant)
* Customer country of residence
* Customer state of residence
* NPS score (satisfaction level 0-10 from questionnaire)
* Time to first success moment (days)
* Total number of times logged in
* Time since last login
* Days since key inflection points (work with SME in marketing/product) this could be days since logging in, since getting their first result from the technology, etc.
* Time spent logged in past month
* Time spent logged in average/mo for length of subscription
* Number of times they have contacted customer service over life of subscription
* Number of customer service calls in the prior month that they cancelled
* Number of times they opened and clicked on the Help text in the app or online
* What they typed into the search box in the help text in the app
* Tenure (Lifetime account duration in days)
* Total subscription amount paid
* Date unsubscribed (timestamp)
* How many other products they have from our company
* What other products they have from our company, as a separate column for each (yes / no)
* Anything else specific to the business of note
* Cancelled yes or no (Churn)

## 3: SetUp: Importing the Libraries

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms
from sklearn import model_selection, metrics  #to include metrics for evaluation # this used to be cross_validation
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
%matplotlib inline

In [62]:
# Quiet warnings since this is a demo (it quiets future and deprecation warnings).
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

## 4: Access Data and Clean It Up

We're using the IBM Telco sample dataset for this case study. It provides some of the datapoints from our wish list but not all.

I staged the IBM sample dataset on my own S3 bucket on AWS and I'm accessing it from there:

In [63]:
!wget https://s3.amazonaws.com/ibm-sample-cust-churn-data/WA_Fn-UseC_-Telco-Customer-Churn.csv

--2025-03-05 09:58:48--  https://s3.amazonaws.com/ibm-sample-cust-churn-data/WA_Fn-UseC_-Telco-Customer-Churn.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 3.5.17.29, 16.182.67.104, 52.217.74.86, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|3.5.17.29|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2025-03-05 09:58:48 ERROR 404: Not Found.



In [64]:
ls -l

total 4
drwxr-xr-x 1 root root 4096 Mar  3 14:19 sample_data/


In [65]:
# Read the data and view the top portion to see what we are dealing with.
data=pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'WA_Fn-UseC_-Telco-Customer-Churn.csv'

In [ ]:
# See if the data is usable.
data.info()

In [ ]:
# Analyze if there is non-numeric data in the 'TotalCharges' column since it's showing as an object instead of float64.
data['TotalCharges'] = pd.to_numeric(data['TotalCharges'], errors = 'coerce')
data.loc[data['TotalCharges'].isna()==True]

In [ ]:
# Above we see that the blank "TotalCharges" happen when customers have 0 months tenure so we will change those values to $0.
data[data['TotalCharges'].isna()==True] = 0
data['OnlineBackup'].unique()

In [ ]:
# See how many rows and columns.
data.shape

More data cleanup: next we’ll convert the categorical values into numeric values.

In [ ]:
data['gender'].replace(['Male','Female'],[0,1],inplace=True)
data['Partner'].replace(['Yes','No'],[1,0],inplace=True)
data['Dependents'].replace(['Yes','No'],[1,0],inplace=True)
data['PhoneService'].replace(['Yes','No'],[1,0],inplace=True)
data['MultipleLines'].replace(['No phone service','No', 'Yes'],[0,0,1],inplace=True)
data['InternetService'].replace(['No','DSL','Fiber optic'],[0,1,2],inplace=True)
data['OnlineSecurity'].replace(['No','Yes','No internet service'],[0,1,0],inplace=True)
data['OnlineBackup'].replace(['No','Yes','No internet service'],[0,1,0],inplace=True)
data['DeviceProtection'].replace(['No','Yes','No internet service'],[0,1,0],inplace=True)
data['TechSupport'].replace(['No','Yes','No internet service'],[0,1,0],inplace=True)
data['StreamingTV'].replace(['No','Yes','No internet service'],[0,1,0],inplace=True)
data['StreamingMovies'].replace(['No','Yes','No internet service'],[0,1,0],inplace=True)
data['Contract'].replace(['Month-to-month', 'One year', 'Two year'],[0,1,2],inplace=True)
data['PaperlessBilling'].replace(['Yes','No'],[1,0],inplace=True)
data['PaymentMethod'].replace(['Electronic check', 'Mailed check', 'Bank transfer (automatic)','Credit card (automatic)'],[0,1,2,3],inplace=True)
data['Churn'].replace(['Yes','No'],[1,0],inplace=True)

data.info()

Let's look at relationships between customer data and churn using correlation.

In [ ]:
corr = data.corr()
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, annot = True, annot_kws={'size':12})
heat_map=plt.gcf()
heat_map.set_size_inches(20,15)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

Our goal is to avoid multicollinearity by dropping features that are closely correlated with each other. For example here it is TotalCharges and MonthlyCharges. So we will drop TotalCharges.

In [ ]:
	data.pop('TotalCharges')

In [ ]:
# Run info again to make sure TotalCharges has been dropped (popped off).
data.info()

Rerun corr chart after cleanup. TotalCharges should not appear in the corr chart.

In [ ]:
corr = data.corr()
sns.heatmap(corr, xticklabels=corr.columns.values, yticklabels=corr.columns.values, annot = True, annot_kws={'size':12})
heat_map=plt.gcf()
heat_map.set_size_inches(20,15)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.show()

## 5: Explore The Data

In [ ]:
# Explore how many churn data points we have.
print(len(data['Churn']))

In [ ]:
# Explore how many customers in this dataset have churned. Is this dataset 50% as the team suggests is the overall customer churn rate?
data['Churn'].value_counts()
# We see this dataset actually has less than the overall 50% churn rate of the entire company reported data (it's actually 26.54% that have churned.

In [ ]:
# This creates a bar graph of churn (Yes vs. No) so we can check how the data is balanced.
data['Churn'].value_counts().plot(kind = 'bar', title = 'Bar Graph of Non-Churners vs Churners by Count (Churn is a 1)', color = 'blue', align = 'center')
plt.show()
# The dataset does not have a huge imbalance which is good news! But also we clearly see it does not have the 50% as we would have thought.

Explore some contingencies on how some features relate to churn.

In [ ]:
# Creates initial contingency table between Churn and gender. Male is 0, Female is 1.
gender_churn_contingency = pd.crosstab(data["gender"], data["Churn"])
display(gender_churn_contingency)
# Male and females churn at about the same rate, so not much to see here. Let's keep moving.

In [ ]:
# Check the data health. The sections should all be completely black indicating the data is complete.
ms.matrix(data)
# It looks good.

In [ ]:
# Explore the relationship between instances of Tech Support and Churn.
# Stacked Bar of Tech Support and Churn.
tech_support_churn = pd.crosstab(data['TechSupport'], data['Churn'])
tech_support_churn.plot(kind = 'bar', stacked = True)
plt.ylabel('Count')
plt.xlabel('Tech Support Count')
plt.title('Churn Rate Relative to Uses of Tech Support (Churned is a 1)')
plt.show()
# We can see that non-churners use tech support more often than customers that end up churning.
# So let's explore some ways to get people to use Tech Support more often so they cancel (churn) less. You can see notes for this at the bottom.
# Also, tech support in this data is just a Y/N. It would be useful in future to include how many tech support calls by customer so we could analyze how the number of tech support calls relates to churn.

In [ ]:
# Churn rate relative to tenure.
# Stacked bar of tenure and churn.
tenure_churn = pd.crosstab(data['tenure'], data['Churn'])
tenure_churn.plot(kind = 'bar', stacked = True)
plt.ylabel('Count')
plt.xlabel('Tenure of Subscription')
plt.title('Churn Rate Relative to Tenure of Subscription (Churned is a 1)')
plt.show()
# We can clearly see the longer a customer stays as a subscriber, the less they are likely to churn!

In [ ]:
# Distribution of features.
features = ['gender', 'SeniorCitizen','Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges']
data[features].describe()

In [ ]:
# Plot the distribution of observations for tenure.
sns.distplot(data['tenure']);
# It shows the max tenure is 70. This must be when the data history ends. We'll account for this in our analysis.

In [ ]:
# Does how a customer pays have to do with their churn?
_, axes = plt.subplots(1, 2, sharey=True, figsize=(10, 4))
sns.countplot(x='PaperlessBilling', hue='Churn',
              data=data, ax=axes[0]);
sns.countplot(x='PaymentMethod', hue='Churn',
              data=data, ax=axes[1]);
# We can see that customers that use paperless billing are much more likely to churn (0 = don't have paperless billing). That seems backwards I would go check that data with the team.
# We can see that customers that have the 0 payment method (electronic check) are much more likely to churn. Let's discourage that option.

In [ ]:
# See if the other products they have from this company has to do with their churn.
_, axes = plt.subplots(1, 2, sharey=True, figsize=(10, 4))
sns.countplot(x='PhoneService', hue='Churn',
              data=data, ax=axes[0]);
sns.countplot(x='InternetService', hue='Churn',
              data=data, ax=axes[1]);
# If they don't have Phone Service, they are more likely to churn.
# If they don't have Internet Service, they are more likely to churn. Those customers with the highest Internet Service are least likely to churn.
# Conclusion: This makes sense. Customers with other products from the company, and premium products, churn less.
# Offer customers these additional products, perhaps even at a deep discount, so they take them and are less likely to churn.

## 6:  Prepare the Data

In [ ]:
# Splitting the data for testing and training.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('Churn',axis=1),
                                                    data['Churn'], test_size=0.30,
                                                    random_state=101)

In [ ]:
train=pd.concat([X_train,y_train],axis=1)

In [ ]:
# Function to estimate the best value of n_estimators and fit the model with the given data.

In [ ]:
def modelfit(alg, dtrain, predictors,useTrainCV=True, cv_folds=5, early_stopping_rounds=50):

    if useTrainCV:
        #to get the parameters of xgboost
        xgb_param = alg.get_xgb_params()

        #to convert into a datastructure internally used by xgboost for training efficiency
        # and speed
        xgtrain = xgb.DMatrix(dtrain[predictors].values, label=dtrain[target].values)

        #xgb.cv is used to find the number of estimators required for the parameters
        # which are set
        cvresult = xgb.cv(xgb_param, xgtrain,
                          num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
                        metrics='auc', early_stopping_rounds=early_stopping_rounds)

        #setting the n_estimators parameter using set_params
        alg.set_params(n_estimators=cvresult.shape[0])

        print(alg.get_xgb_params())

    #Fit the algorithm on the data
    alg.fit(dtrain[predictors], dtrain['Churn'],eval_metric='auc')

    return alg

In [ ]:
# Function to get the accuracy of the model on the test data given the features considered.

In [ ]:
def get_accuracy(alg,predictors):
    dtrain_predictions = alg.predict(X_test[predictors])
    dtrain_predprob = alg.predict_proba(X_test[predictors])[:,1]
    print ("\nModel Report")
    print ("Accuracy : %.4g" % metrics.accuracy_score(y_test.values,
                                                      dtrain_predictions))
    print ("AUC Score (Train): %f" % metrics.roc_auc_score(y_test.values,
                                                           dtrain_predprob))

In [ ]:
# Function to get the feature importances based on the model fit.

In [ ]:
def get_feature_importances(alg):
    #to get the feature importances based on xgboost we use fscore
    feat_imp = pd.Series(alg._Booster.get_fscore()).sort_values(ascending=False)
    print(feat_imp)

    #this shows the feature importances on a bar chart
    feat_imp.plot(kind='bar', title='Feature Importances')
    plt.ylabel('Feature Importance Score')

In [ ]:
target = 'Churn'
IDcol = 'customerID'

## 7: Model Selection, Predictions, and Metrics

In [ ]:
# To return the XGBClassifier object based on the values of the features.

In [ ]:
!pip install xgboost
# XGBoost converts weak learners to strong learners through an ensemble method.
# Unlike bagging, in the classical boosting the subset creation is not random and depends upon the performance of the previous models.

In [ ]:
def XgbClass(learning_rate =0.1,n_estimators=1000,max_depth=5,min_child_weight=1,
             gamma=0,subsample=0.8,colsample_bytree=0.8):
    xgb1 = XGBClassifier(learning_rate=learning_rate,
                         n_estimators=n_estimators,
                         max_depth=max_depth,
                         min_child_weight=min_child_weight,
                         gamma=gamma,
                         subsample=subsample,
                         colsample_bytree=colsample_bytree)
    return xgb1

In [ ]:
# Function to return the list of predictors.

In [ ]:
# These are the initial parameters before tuning.
def drop_features(l):
    return [x for x in train.columns if x not in l]

### First Prediction: Use of initial parameters and without feature engineering

In [ ]:
from xgboost import XGBClassifier
import xgboost as xgb

In [ ]:
predictors = drop_features([target, IDcol])
xgb1=XgbClass()
first_model=modelfit(xgb1, train, predictors)
xgb1.fit(train[predictors],train['Churn'])

In [ ]:
get_accuracy(first_model,predictors)

**Accuracy** is the proportion of true positives and negatives in the whole data set. It determines if a value is accurate compare it to the accepted value; the nearness of a calculation to the true value.

**AUC** (area under the ROC receiver operating characteristic curve) measures how true positive rate (recall) and false positive rate trade off, so in that sense it is already measuring something else. More importantly, AUC is not a function of threshold. It is an evaluation of the classifier as threshold varies over all possible values. It is in a sense a broader metric, testing the quality of the internal value that the classifier generates and then compares to a threshold.

**Accuracy vs AUC**: The accuracy depends on the threshold chosen, whereas the AUC considers all possible thresholds. Because of this it is often preferred as it provides a “broader” view of the performance of the classifier, but they still measure different things and as such using one or the other is problem-dependent.

In [ ]:
get_feature_importances(first_model)

### Second Prediction: Using intial Parameters and removing features of least importances

In [ ]:
# Model after removing the features of least importance.

In [ ]:
dropl=['DeviceProtection','Dependents','Dependents','gender','StreamingMovies','MultipleLines']

In [ ]:
dropl_first=dropl+[target,IDcol]

In [ ]:
# These are the initial parameters before tuning.
predictors = drop_features(dropl_first)
xgb1 = XgbClass()
second_model=modelfit(xgb1, train, predictors)

In [ ]:
get_accuracy(second_model,predictors)

In [ ]:
get_feature_importances(second_model)

### Third Prediction: Again removing the features of least importance

In [ ]:
dropl1=dropl+['Partner','PhoneService','OnlineBackup','TechSupport','OnlineSecurity']
dropl_second=dropl_first+['Partner','PhoneService','OnlineBackup','TechSupport','OnlineSecurity']

In [ ]:
predictors=drop_features(dropl_second)

In [ ]:
xgb1=XgbClass()
third_model=modelfit(xgb1,train,predictors)

In [ ]:
get_accuracy(third_model,predictors)

## 8: Predict on New Cases

In [ ]:
# Function stores the result in required csv file and features. I removed this section for now as the csv files are not yet saving correctly. See prototype version of notebook for details.

## 9: Tuning

In [ ]:
# Tune max_depth and min_child_weight.

In [ ]:
predictors = drop_features(dropl_first)
predictors

In [ ]:
param_test1 = {
 'max_depth':list(range(5,10,1)),
 'min_child_weight':list(range(5,10,1))
}
gsearch1 = GridSearchCV(estimator=XgbClass(n_estimators=48),param_grid =param_test1,
                        scoring='roc_auc',n_jobs=-1,iid=False, cv=5, verbose=3)
gsearch1.fit(train[predictors],train[target])
gsearch1.cv_results_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
# If the best parameters are edge values then we do
# GridSearchCV by taking one less and one value more than the best parameters.

In [ ]:
param_test2 = {
 'max_depth':[6,7,8,9],
 'min_child_weight':[2,3,4,5]
}
gsearch2 = GridSearchCV(estimator=XgbClass(n_estimators=48),param_grid =param_test2,scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch2.fit(train[predictors],train[target])
gsearch2.cv_results_, gsearch2.best_params_, gsearch2.best_score_

In [ ]:
xgb1 = XgbClass(max_depth=8,min_child_weight=4)
model=modelfit(xgb1, train, predictors)

In [ ]:
get_accuracy(model,predictors)

In [ ]:
# To tune gamma.

In [ ]:
param_test3 = {
 'gamma':[i/10.0 for i in range(0,8)]
}
gsearch3=GridSearchCV(estimator=XgbClass(n_estimators=48,max_depth=7,min_child_weight=5),
                      param_grid =param_test3,scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch3.fit(train[predictors],train[target])
gsearch3.cv_results_, gsearch3.best_params_, gsearch3.best_score_

In [ ]:
xgb1 = XgbClass(max_depth=7,min_child_weight=5,gamma=0)
model=modelfit(xgb1, train, predictors)

In [ ]:
get_accuracy(model,predictors)

In [ ]:
param_test4 = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4=GridSearchCV(estimator=XgbClass(n_estimators=48,max_depth=7,
                                         min_child_weight=5,gamma=0),
                      param_grid =param_test4,scoring='roc_auc',n_jobs=-1,iid=False, cv=5)
gsearch4.fit(train[predictors],train[target])
gsearch4.cv_results_, gsearch4.best_params_, gsearch4.best_score_

In [ ]:
xgb1 = XgbClass(max_depth=8,min_child_weight=4,gamma=0.4,subsample=0.8,colsample_bytree=0.6)
model=modelfit(xgb1, train, predictors)

In [ ]:
get_accuracy(model,predictors)

In [ ]:
#dropl1=dropl+['Partner','PhoneService','OnlineBackup','TechSupport','OnlineSecurity']
# RunTestAndSaveResults(dropl,'final_result.csv',model)
#FixMeLater the final_result file is not printing to csv

In [ ]:
ls -l

## 10: Next Steps
* Run some actual data from your company. For case study purposes, let's keep going with dummy data ...
* After some initial analysis, cleaning, and tuning we're consistently getting an accuracy of 80% and AUC ~84%. Feature engineering and tuning did not improve (in some cases made worse, seems to be overfitting).
* This meets our prediction success threshold of 80%.
* Share findings with team.
* Discuss some possible next steps:
    * Get more data. This is only ~7k records and much different than the stated overall 50%.
    * Get more detailed data on features that are showing to be relatively MORE predictive:
      * MonthlyCharges
      * Tenure
      * PaymentMethod
      * Plan Types
      
  * **Get more detailed data on features that are showing to be relatively LESS predictive but our analysis show with more details they could be really useful**
      * Tech support: how many calls to tech support lifetime and in past month since churning, instead of just a contact tech support: Y / N.
  * **Get additional data on features we think may be very predictive but we don't have here yet**
    * Time to first success point
    * Other data specific to the business
    * Etc.
    
 ## Help Transform Customer Experience
  * Iterate products to transform customer experiences:
    * Personalized features (i.e if low NPS go to different flow)
    * Data predictions (i.e. predict churn from TechSupport interaction and alert the customer support team to intervene, and automatically have the technology prompt the user for answering their questions),
    * Propensity modeling (i.e set scorecards for customer predicted churn based on numerous factors. flag those users when they go 'red" for action with marketing and customer case outreach).
  
  **Activity loop:**
  * Iterate
  * Communicate
  * Put into production
  * Scale
  * Assess
  * Repeat
  
  Questions?
  Michelle Bonat
  michelle.bonat@gmail.com
  [michellebonat.com](http://michellebonat.com)

## 11: Appendix

## The initial data showed that more tech support interactions reduced customer churn. Here are some ways to put that information into action:
To get customers interacting with tech support instead of cancelling:

1) Include this NPS data in the dataset for analysis, it is from the questionnaire in QuickBooks.

2) Make sure screens/flow to include links to Support during the Cancel process.

3) Offer users a way to get help even live chat right there instead of just a cancel option.

4) There could also be a 'Resolve Your Issue' link right next to the cancel option.